In [4]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt
import scipy.stats

Idea:
    JUNTAR TOTS ELS DATASETS EN 1 DE SOL AMB UN NOU ATRIBUT QUE SIGUI LA DECADA. Deixar la ultima o ultimes decades per dades de test. o si no agafar tant percent de cada decada per test. Primer probare agafant les decades de 60 fins a 90 per a train i deixare el 2000 i 2010 com a test.    

In [15]:
# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

datasets_to_test = ["00", "10"]  # decada 2000 i 2010
dataset_names_to_join = ["60", "70", "80" , "90"] # decada 1960, 1970, 1980 i 1990

temp_ds = []
# Carreguem dataset d'exemple
for i in range(len(dataset_names_to_join)):
    dataset = load_dataset('data/dataset-of-{0}s.csv'.format(dataset_names_to_join[i]))
    dataset['decade'] = dataset_names_to_join[i]
    temp_ds.append(dataset)

    # data = dataset.values

dataset_test = pd.concat(temp_ds, axis=0, ignore_index=True)

dataset_test.info()

dataset_test.drop(["track", "artist", "uri"],axis=1,inplace=True)

dataset_test.to_csv('data/dataset_test_joined.csv')


data = dataset_test.values
target = dataset_test.target

x = data[:, :18]
y = target

print("Dimensionalitat de la BBDD:", dataset.shape)
print("Dimensionalitat de les entrades X", x.shape)
print("Dimensionalitat de l'atribut Y", y.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28836 entries, 0 to 28835
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track             28836 non-null  object 
 1   artist            28836 non-null  object 
 2   uri               28836 non-null  object 
 3   danceability      28836 non-null  float64
 4   energy            28836 non-null  float64
 5   key               28836 non-null  int64  
 6   loudness          28836 non-null  float64
 7   mode              28836 non-null  int64  
 8   speechiness       28836 non-null  float64
 9   acousticness      28836 non-null  float64
 10  instrumentalness  28836 non-null  float64
 11  liveness          28836 non-null  float64
 12  valence           28836 non-null  float64
 13  tempo             28836 non-null  float64
 14  duration_ms       28836 non-null  int64  
 15  time_signature    28836 non-null  int64  
 16  chorus_hit        28836 non-null  float6

In [6]:
print("Per comptar el nombre de valors no existents:")
print(dataset.isnull().sum())

Per comptar el nombre de valors no existents:
track               0
artist              0
uri                 0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
chorus_hit          0
sections            0
target              0
decade              0
dtype: int64


In [7]:
print("Per comptar el nombre de cancons per artistes:")
print(x[:,0])

Per comptar el nombre de cancons per artistes:
['Jealous Kind Of Fella' 'Initials B.B.' 'Melody Twist' ... 'Good Times'
 'Inane' 'You Can Make History (Young Again)']


In [8]:
print("Per visualitzar les primeres 5 mostres de la BBDD:")
dataset.head() 

Per visualitzar les primeres 5 mostres de la BBDD:


,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,decade
0,Misty Roses,Astrud Gilberto,spotify:track:50RBM1j1Dw7WYmsGsWg9Tm,0.527,0.316,1,-15.769,1,0.031,0.693,0.007,0.168,0.543,116.211,158840,4,53.895,6,0,90
1,Never Ever,All Saints,spotify:track:5FTz9qQ94PyUHETyAyfYZN,0.738,0.541,1,-5.485,1,0.031,0.559,0.000,0.049,0.309,134.187,387573,4,32.169,16,1,90
2,Soul Sermon,Gregg Karukas,spotify:track:6m24oe3lk1UMxq9zq4iPFi,0.736,0.419,0,-10.662,1,0.030,0.693,0.495,0.081,0.265,93.982,237267,4,42.054,9,0,90
3,Clarinet Marmalade - Live,Alton Purnell,spotify:track:5FOXuiLI6knVtgMUjWKj6x,0.565,0.594,5,-13.086,1,0.065,0.655,0.926,0.675,0.763,114.219,375933,4,80.997,10,0,90
4,До смерті і довше - Drum & Base and Rock Remix,Skryabin,spotify:track:6CxyIPTqSPvAPXfrIZczs4,0.513,0.760,4,-10.077,1,0.035,0.000,0.003,0.153,0.961,153.166,430653,4,25.573,20,0,90


Primer vaig probar diferents moddels per veure quin hem donava millor resultat.

In [10]:
models = {
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Net": MLPClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    print("###")
    print(name)
    print("###")
    model.fit(x_train, y_train)
    print(name + " trained.")

for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(x_test, y_test) * 100))
    print("###")
    print("Model evaluation {0}".format(name))
    print("###")
    evaluate_model(y_test, model.predict(x_test))
    print("###")
    print("Cross validation {0}".format(name))
    print("###")
    cross_val_preds = cross_val_predict(model, x_val, y_val)


<IPython.core.display.Javascript object>

Com podem veure, el model de Random Forest ens retorna l'accuracy mes alt. Per tant haurem de treballar amb aquest. El seguent que volia veure era buscar quines variables ens ajuden a diferenciar entre HIT i no HIT.

In [ ]:
#create two var lists, one with Spotify's features (spfeatures_var_list) and one with the song traits (song_traits_var_list)
spfeatures_var_list = ['danceability', 'energy', 'key', 'loudness','mode', 'speechiness', 'acousticness', 
                       'instrumentalness', 'liveness','valence']
song_traits_var_list = ['key', 'loudness','tempo', 'time_signature', 'chorus_hit','sections'] 
#duration_ms has been removed since it has such larger numbers than the other variables 

all_songs_hits = dataset_train[spfeatures_var_list].loc[dataset_train['target'] == 1]

all_songs_flops = dataset_train[spfeatures_var_list].loc[dataset_train['target'] == 0]

#create a dataframe that includes the means for hits and flops
hits_means = pd.DataFrame(all_songs_hits.describe().loc['mean'])
flops_means = pd.DataFrame(all_songs_flops.describe().loc['mean'])
means_joined = pd.concat([hits_means,flops_means], axis = 1)
means_joined.columns = ['hit_mean', 'flop_mean']

ss = StandardScaler()
means_joined_scaled = pd.DataFrame(ss.fit_transform(means_joined),index= means_joined.index, columns = means_joined.columns)
means_joined_scaled


means_joined_scaled.plot(kind = 'bar', figsize=(10, 5), color = ('purple', 'grey'), title = 'Means of Hit Songs and Flop Songs for Song Features')
plt.legend(labels=['Hits', 'Flops'], loc='upper right')
plt.show()
plt.show()

Anem a veure l'histograma de cada variable per buscar les distribucions de les nostres variables

In [ ]:
#create histograpms of all the variables to see distributions
fig, ax = plt.subplots(5,3, figsize=(5,5))

def hist_plot(row, column, variable, binsnum, color):
    ax[row, column].hist(dataset_train[variable], bins = binsnum, color = color)
    ax[row, column].set_title(variable + ' histogram')
    
hist_plot(0, 0, 'danceability', 10, 'purple')
hist_plot(0, 1, 'energy', 10, 'orchid')
hist_plot(0, 2, 'key', 10, 'plum')
hist_plot(1,0, 'loudness', 10, 'purple')
hist_plot(1,1, 'mode', 10, 'orchid')
hist_plot(1,2, 'speechiness', 10, 'plum')
hist_plot(2,0, 'acousticness', 10, 'purple')
hist_plot(2,1, 'instrumentalness', 10, 'orchid')
hist_plot(2,2, 'liveness', 10, 'plum')
hist_plot(3,0, 'valence', 10, 'purple')
hist_plot(3,1, 'tempo', 10, 'orchid')
hist_plot(3,2, 'duration_ms', 50, 'plum')
hist_plot(4,0, 'time_signature', 10, 'purple')
hist_plot(4,1, 'chorus_hit', 10, 'orchid')
hist_plot(4,2, 'sections', 50, 'plum')

plt.show()

Aquí hi ha alguns patrons interessants: les cançons solen ser més ballables que menys ballables, les cançons tendeixen a tenir més energia que menys energia, la tonalitat de C és la tonalitat més popular, les cançons solen ser inferiors a 10 decibels, la majoria de les cançons estan en escales majors, la majoria de les cançons contenen més música que la parla, la majoria de les cançons no són en directe, la majoria de les cançons no són acústiques, la majoria de les cançons contenen música, hi ha una bona barreja de cançons alegres i tristes, la majoria de cançons tenen uns 80-90 pulsacions per minut i la majoria de cançons són en 4/4 de temps.

Anem a crear un model de random forest i provar rendiments

In [ ]:
RF = RandomForestClassifier()
RF.fit(x_train, y_train)
y_pred = RF.predict(x_test)

#create a confusion matrix to see the efficacy of the model
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

#create a figure/heatmap of the confusion matrix for a better visual
mpl.rcParams['figure.figsize']=(10,5)
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="RdPu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.show()

The confusion matrix demonstrates that the model correctly identified hits and flops most of the time. 
Podem observar com al pas de les decades, com de importants son cara atribut de les cançons ja que possiblement depenent de la decada, li donaven mes importancia potser al tant per cetn de ballable  i no tant al tant percent de instrumentabilitat.


In [ ]:
#create a dataframe of the feature importances to determine which variables are the most important in determining a hit
all_songs_feat = RF.feature_importances_
df_indep_columns = pd.DataFrame(indep_columns)
df_all_songs_feat = pd.DataFrame(all_songs_feat)
all_songs_feat_vars = pd.concat([df_indep_columns, df_all_songs_feat], axis = 1)
all_songs_feat_vars.columns = ['Variable', 'Feature importance all decades']
all_songs_feat_vars = all_songs_feat_vars.set_index('Variable')
all_songs_feat_vars = all_songs_feat_vars.sort_values(by=['Feature importance all decades'], ascending = False)
all_songs_feat_vars
all_songs_feat_vars.to_csv('all_songs_feat.csv', index = False) #create a CSV file of the new dataframe

all_songs_feat_vars.plot(kind='bar', color = "purple", title = "Most important features for predicting hit and flop songs for all decades", legend = None)
plt.ylabel('Feature importance')
plt.show()

For all decades, instrumentalness, danceability, acousticness, duration_ms, and loudness were the greatest predictors of if a song was a hit.

# Proves amb NLP (NLTK)